### instantiating services

In [2]:
from datetime import datetime, timezone
from firebase_admin import initialize_app
from firebase_admin import firestore,credentials
from web3 import Web3
import re
cred = credentials.Certificate('homebase.json')
initialize_app(cred)
db = firestore.client()
some=db.collection("some")
# Always get the current time in UTC
uite = datetime.now(timezone.utc)
some.document("whatever").set({"what": uite})

update_time {
  seconds: 1733756664
  nanos: 768808000
}

In [26]:
daos_collection=db.collection('idaosEtherlink-Testnet')
proposal_doc_ref = daos_collection \
            .document("0xb5e60De29AF240DF386DCF26eA529f477b93f3Ef") \
            .collection('proposals') \
            .document("41296605969268937190325073060614567362649195436429149154767968960542315171563")
ceva=proposal_doc_ref.get()
data=ceva.to_dict()
from apps.homebase.entities import Proposal
org=None
prop:Proposal = Proposal(name="whatever", org=org)
prop.fromJson(data)
print(f"against {prop.against}, inFavor {prop.inFavor}")

against 0, inFavor 1333


In [ ]:
import codecs
from eth_utils import to_bytes

def hex_to_bytes(hex_string):
    return codecs.decode(hex_string.replace("0x", ""), "hex")

def bytes_to_int(byte_array):
    return int.from_bytes(byte_array, byteorder="big")

def decode_function_parameters(function_abi, data_bytes):
    data_without_selector = data_bytes[4:]
    offset = 0
    decoded_params = []
    for param in function_abi["inputs"]:
        param_type = param["type"]
        if param_type == "string":
            # Decode string (dynamic)
            param_offset = bytes_to_int(data_without_selector[offset:offset + 32])
            string_offset = param_offset
            string_length = bytes_to_int(data_without_selector[string_offset:string_offset + 32])
            string_data = data_without_selector[string_offset + 32:string_offset + 32 + string_length]
            decoded_params.append(string_data.decode("utf-8"))
        elif param_type == "address":
            # Decode address (last 20 bytes of the 32-byte slot)
            address_bytes = data_without_selector[offset + 12:offset + 32]
            decoded_params.append("0x" + address_bytes.hex())
        elif param_type.startswith("uint"):
            # Decode uint (entire 32 bytes)
            uint_bytes = data_without_selector[offset:offset + 32]
            decoded_params.append(str(bytes_to_int(uint_bytes)))
        else:
            raise ValueError(f"Unsupported parameter type: {param_type}")

        # Move to the next 32-byte slot
        offset += 32

    return decoded_params


In [ ]:
mint_function_abi = {
    "name": "mint",
    "inputs": [
        {"name": "to", "type": "address"},
        {"name": "amount", "type": "uint256"},
    ],
}

In [27]:
decoded = decode_function_parameters(mint_function_abi, prop.callDatas[0])

In [28]:
print(decoded)
prop.targets

['0xa6a40e0b6db5a6f808703dbe91dbe50b7fc1fa3e', '32100']


['0xB14179168ec05a77423cD3d7bB68D14a81187bFF']

In [38]:
Web3.to_checksum_address("0xa6a40e0b6db5a6f808703dbe91dbe50b7fc1fa3e")

'0xA6A40E0b6DB5a6f808703DBe91DbE50B7FC1fa3E'

In [9]:
from apps.homebase.abis import tokenAbiGlobal
from web3 import Web3
from datetime import datetime
rpc="https://node.ghostnet.etherlink.com"
web3 = Web3(Web3.HTTPProvider(rpc))
block=web3.eth.get_block('latest')
now=datetime.now()
timestamp =block['timestamp']
dt_object = datetime.utcfromtimestamp(timestamp)
print(f"from chain: {dt_object.strftime('%Y-%m-%d %H:%M:%S')}")
print("timestamp from chain "+str(timestamp))
print(f"local: {now.strftime('%Y-%m-%d %H:%M:%S')}")
token_contract = web3.eth.contract(
    address="0x5B78FE1Fd6e42786183A40F8e471Fb319F362235", abi=tokenAbiGlobal)
balance = token_contract.functions.balanceOf(Web3.to_checksum_address("0xa6a40e0b6db5a6f808703dbe91dbe50b7fc1fa3e")).call()

from chain: 2024-12-15 13:09:16
timestamp from chain 1734268156
local: 2024-12-15 15:09:19


C:\Users\Administrator\AppData\Local\Temp\ipykernel_24608\1181815313.py:9: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_object = datetime.utcfromtimestamp(timestamp)


In [12]:
supply=token_contract.functions.totalSupply().call()
supply

3201000

In [13]:
balance

3200000

In [29]:
ceva=proposal_doc_ref.get()
option=0
votingWeight="400000"
if option==1:
    prop.inFavor= str(int(prop.inFavor)+int(votingWeight))
    prop.votesFor+=1
elif option==0:
    prop.against= str(int(prop.against)+int(votingWeight))
    prop.votesAgainst+=1
proposal_doc_ref.set(prop.toJson())


update_time {
  seconds: 1733579362
  nanos: 147842000
}

In [ ]:
from apps.homebase.abis import wrapperAbi, daoAbiGlobal, tokenAbiGlobal
from apps.homebase.paper import Paper
from datetime import datetime, timezone
import time
from firebase_admin import initialize_app
from firebase_admin import firestore, credentials
from web3 import Web3
import os
import sys
import re
cred = credentials.Certificate('homebase.json')
initialize_app(cred)
db = firestore.client()
networks = db.collection("contracts")
ceva = networks.document("Etherlink-Testnet").get()
wrapper_address = ceva.to_dict()['wrapper']
print("wrapper address :" + str(wrapper_address))
rpc = "https://node.ghostnet.etherlink.com"

web3 = Web3(Web3.HTTPProvider(rpc))
papers = {}
daos = []
if web3.is_connected():
    print("node connected")
else:
    print("node connection failed!")

daos_collection = db.collection('idaosEtherlink-Testnet')
docs = list(daos_collection.stream())
dao_addresses = [doc.id for doc in docs]

for doc in docs:
    obj = doc.to_dict()
    try:
        p = Paper(address=obj['token'], kind="token",
                  daos_collection=daos_collection, db=db,  web3=web3, dao=doc.id)
        dao = Paper(address=obj['address'], kind="dao",token=p,
                    daos_collection=daos_collection, db=db,  web3=web3, dao=doc.id)
    except Exception as e:
        print("one DAO contract can't parse correctly: "+str(e))
    papers.update({obj['token']: p})
    papers.update({obj['address']: dao})

event_signatures = {
    web3.keccak(text="NewDaoCreated(address,address,address[],uint256[],string,string,string,uint256,address,string[],string[])").hex(): "NewDaoCreated",
    "0x01c5013cf023a364cc49643b8f57347e398d2f0db0968edeb64e7c41bf2dfbde": "NewDaoCreated",
    "0x3134e8a2e6d97e929a7e54011ea5485d7d196dd5f0ba4d4ef95803e8e3fc257f": "DelegateChanged",
    "0x7d84a6263ae0d98d3329bd7b46bb4e8d6f98cd35a7adb45c274c8b7fd5ebd5e0": "ProposalCreated",
    "0x9a2e42fd6722813d69113e7d0079d3d940171428df7373df9c7f7617cfda2892": "ProposalQueued",
    "0x712ae1383f79ac853f8d882153778e0260ef8f03b504e2866e0593e04d2b291f": "ProposalExecuted",
    "0xb8e138887d0aa13bab447e82de9d5c1777041ecd21ca36ba824ff1e6c07ddda4": "VoteCast"
}

papers.update({wrapper_address: Paper(address=wrapper_address,
              kind="wrapper", daos_collection=daos_collection, db=db,  web3=web3)})
listening_to_addresses = [wrapper_address]
listening_to_addresses = listening_to_addresses+list(papers.keys())

counter = 0
processed_transactions = set()
heartbeat = 0
print(f"Listening for {len(event_signatures)} events on {
      len(papers.items())} contracts...")

while True:
    heartbeat += 1
    try:
        latest = web3.eth.block_number
        first = latest-13
        logs = web3.eth.get_logs({
            "fromBlock": first,
            "toBlock": latest,
            "address": listening_to_addresses,
            # "topics": [[*event_signatures.keys()], None]
        })
        for log in logs:
            tx_hash = log["transactionHash"].hex()
            if tx_hash in processed_transactions:
                print("already did this one")
                continue  # Skip duplicate
            contract_address = log["address"]
            event_signature = "0x"+log["topics"][0].hex()
            if event_signatures.get(event_signature) is not None:
                event_name = event_signatures[event_signature]
                processed_transactions.add(tx_hash)
            else:
                notfound = True
                print("not found")
                continue
            event_name = event_signatures[event_signature]
            print(f"Event: {event_name}, Contract: {contract_address}")
            print(f"Event: {event_name}, Contract: {contract_address}")
            new_contract_addresses = papers[contract_address].handle_event(
                log, func=event_name)
            if new_contract_addresses != None:
                dao_address = new_contract_addresses[0]
                token_address = new_contract_addresses[1]
                print("adding dao "+dao_address+" and token "+token_address)
                listening_to_addresses = listening_to_addresses + \
                    [dao_address] + [token_address]
                print("latest addresses added " +
                      str(listening_to_addresses[-1]+", "+str(listening_to_addresses[-2])))
                papers.update({token_address: Paper(
                    address=token_address, kind="token", daos_collection=daos_collection, db=db, dao=dao_address, web3=web3)})
                papers.update({dao_address: Paper(
                    address=dao_address, kind="dao", daos_collection=daos_collection, db=db, dao=dao_address, web3=web3)})

    except Exception as e:
        print("something went wrong "+str(e))
        web3 = Web3(Web3.HTTPProvider(rpc))
        if web3.is_connected():
            print("node connected")
        else:
            print("node connection failed!")
            os.execl(sys.executable, sys.executable, *sys.argv)
    if heartbeat % 50 == 0:
        print("heartbeat: "+str(heartbeat))

    time.sleep(3)


In [2]:
contract=papers["0x51fD80a11203a73da29519A2B8413C953F5007a9"].get

AttributeError: 'Paper' object has no attribute 'get'

In [3]:
contract= papers['0x096947d75e6e10AFa93587FB662070fB6Dfe521a'].get_token_contract()

In [5]:
totalSupply = contract.functions.totalSupply().call()
totalSupply

89811065